# SQL query examples
_____
### Here are a few of the queries I have worked with

*the examples are written in PostgreSQL*

___

based on the following tables:

**country**

| id | name | 

$~$

**match**

| id | country_id | season | stage | date | hometeam_id | awayteam_id | home_goal | away_goal |



The goal of this query is to generate a subquery using the match table, and then join that subquery to the country table to calculate information about matches with 10 or more goals in total!

### Joining Subqueries in FROM


In [ ]:
SELECT
	-- Select country name and the count match IDs
    c.name AS country_name,
    COUNT(sub.id) AS matches
FROM country AS c
-- Inner join the subquery onto country
-- Select the country id and match id columns
INNER JOIN (SELECT country_id, id 
            FROM match
            -- Filter the subquery by matches with 10+ goals
            WHERE (home_goal + away_goal) >= 10) AS sub
ON c.id = sub.country_id
GROUP BY country_name;

It generated a count of matches in each country where the total goals was higher than 10. 

### Building on Subqueries in FROM



 find out some more details about those matches -- when they were played, during which seasons, and how many of the goals were home versus away goals.

In [ ]:
SELECT
	-- Select country, date, home, and away goals from the subquery
    country,
    date,
    home_goal,
    away_goal
FROM 
	-- Select country name, date, home_goal, away_goal, and total goals in the subquery
	(SELECT name AS country, 
     	    m.date, 
     		m.home_goal, 
     		m.away_goal,
           (m.home_goal + m.away_goal) AS total_goals
    FROM match AS m
    LEFT JOIN country AS c
    ON m.country_id = c.id) AS subq
-- Filter by total goals scored in the main query
WHERE total_goals >= 10;

### Add a subquery to the SELECT clause



With a subquery in select, you can compare each league's average to the overall average

In [ ]:
SELECT 
	l.name AS league,
    -- Select and round the league's total goals
    ROUND(AVG(M.home_goal + m.away_goal), 2) AS avg_goals,
    -- Select & round the average total goals for the season
    (SELECT ROUND(AVG(home_goal + away_goal), 2) 
     FROM match
     WHERE season = '2013/2014') AS overall_avg
FROM league AS l
LEFT JOIN match AS m
ON l.country_id = m.country_id
-- Filter for the 2013/2014 season
WHERE season = '2013/2014'
GROUP BY league;

the result is a table like this:

| league | avg_goals | overall_avg |

### Subqueries in Select for Calculations

$~$

In the previous query, I created a column to compare each league's average total goals to the overall average goals in the 2013/2014 season. In this query, I will add a column that directly compares these values by subtracting the overall average from the subquery.

In [ ]:
SELECT 
	-- Select the league name and average goals scored
	l.name AS league,
	ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
    -- Subtract the overall average from the league average
	ROUND(AVG(m.home_goal + m.away_goal) - 
          (SELECT AVG(home_goal + away_goal)
           FROM match 
    -- filter subquery
           WHERE season = '2013/2014'),2) AS diff
FROM league AS l
LEFT JOIN match AS m
ON l.country_id = m.country_id
-- Filter the main query only include 2013/2014 results
WHERE m.season = '2013/2014'
GROUP BY l.name;

the result is a table like this:

|league| avg_goals| diff
